In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3, VGG16, InceptionResNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GRU, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve, f1_score, recall_score, precision_score
import matplotlib.pyplot as plt
import itertools
import os
import random

In [ ]:
# MPA algoritması
def mpa_optimizer(hyperparams, fitness_function, num_iterations=50):
    num_agents = len(hyperparams)
    best_solution = None
    best_fitness = float('inf')

    # Rastgele başlangıç populasyonu
    positions = np.random.uniform(low=0, high=1, size=(num_agents, len(hyperparams[0])))

    for iteration in range(num_iterations):
        for i in range(num_agents):
            # Rastgele bir yeni pozisyon seç
            new_position = np.random.uniform(low=0, high=1, size=(1, len(hyperparams[0])))

            # Yeni pozisyonun fitness değeri
            fitness = fitness_function(new_position[0])
            if fitness < best_fitness:
                best_fitness = fitness
                best_solution = new_position

    return best_solution[0]

In [ ]:
# CSTOA algoritması
def cstoa_optimizer(init_population, fitness_function, num_iterations=50):
    num_agents = len(init_population)
    best_solution = None
    best_fitness = float('inf')

    # Kaotik haritalama ile populasyonu başlat
    positions = init_population * np.random.uniform(size=(num_agents, len(init_population[0])))

    for iteration in range(num_iterations):
        for i in range(num_agents):
            # Yeni pozisyonu güncelleme
            new_position = positions[i] + np.random.randn(*positions[i].shape)
            fitness = fitness_function(new_position)

            # En iyi çözüm güncelle
            if fitness < best_fitness:
                best_fitness = fitness
                best_solution = new_position

    return best_solution

In [ ]:
# CLAHE ile ön işleme
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Hata: {image_path} konumundaki görüntü okunamadı.")
        return None
    image = cv2.resize(image, (224, 224))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    image = clahe.apply(image)
    image = image / 255.0
    image = np.stack([image, image, image], axis=-1)
    return image

In [ ]:
# Veriyi yükleme ve ön işleme
def load_data(base_dir):
    classes = ['Dense', 'Medium_Dense', 'Sparse', 'None']
    image_paths = []
    labels = []

    for cls in classes:
        class_dir = os.path.join(base_dir, cls)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            processed_image = preprocess_image(img_path)
            if processed_image is not None:
                image_paths.append(processed_image)
                labels.append(cls)

    data = np.array(image_paths)
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    return data, labels

In [ ]:
# Model oluşturma (Inception v3 + GRU)
def build_model(hyperparams, base_model_choice='InceptionV3'):
    if base_model_choice == 'InceptionV3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif base_model_choice == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif base_model_choice == 'InceptionResNetV2':
        base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    else:
        raise ValueError("Geçersiz model seçimi")

    base_model.trainable = False

    model = Sequential([
        base_model,
        Flatten(),
        Dense(int(hyperparams[0]), activation='relu'),
        Reshape((1, int(hyperparams[0]))),
        GRU(max(1, int(hyperparams[1])), return_sequences=True),
        GRU(max(1, int(hyperparams[2]))),
        Dense(4, activation='softmax')
    ])

    learning_rate = hyperparams[3]
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Hiperparametre optimizasyonu
def fitness_function(hyperparams):
    model = build_model(hyperparams)
    model, _ = train_model(model, data, labels, epochs=3)
    loss, accuracy = model.evaluate(data, labels)
    return 1 - accuracy

In [ ]:
# Modeli eğitme
def train_model(model, data, labels, epochs=100, callbacks=None):
    X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_data=(X_val, y_val), callbacks=callbacks)
    return model, history

In [ ]:
# Modeli değerlendirme
def evaluate_model(model, data, labels):
    predictions = model.predict(data)
    predictions = np.argmax(predictions, axis=1)
    report = classification_report(labels, predictions, target_names=['Dense', 'Medium Dense', 'Sparse', 'None'])
    f1 = f1_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    print(report)
    print(f'F1 Score: {f1}, Recall: {recall}, Precision: {precision}')

In [ ]:
# Grafik çizimleri (karışıklık matrisi, doğruluk/kayıp eğrileri, PR ve ROC eğrileri)
def plot_metrics(y_true, y_scores, y_pred_classes, history):
    plot_confusion_matrix(y_true, y_pred_classes, classes=['Dense', 'Medium Dense', 'Sparse', 'None'])
    plot_accuracy_and_loss(history)
    plot_pr_curve(y_true, y_scores, class_names=['Dense', 'Medium Dense', 'Sparse', 'None'])
    plot_roc_curve(y_true, y_scores, class_names=['Dense', 'Medium Dense', 'Sparse', 'None'])

In [ ]:
# Ana program
if __name__ == '__main__':
    base_dir = '/content/drive/MyDrive/Colab Notebooks/dataset/'
    data, labels = load_data(base_dir)

    # Hiperparametre optimizasyonu ve eğitim
    init_population = np.random.uniform(low=[32, 1, 1, 0.0001], high=[256, 128, 64, 0.01], size=(10, 4))
    best_hyperparams_cstoa = cstoa_optimizer(init_population, fitness_function)
    base_model_choices = ['InceptionV3', 'VGG16', 'InceptionResNetV2']

    # Farklı epoch değerleri ve modellerle eğitim ve değerlendirme
    epoch_values = [500, 1000, 1500]
    for model_choice in base_model_choices:
        for epochs in epoch_values:
            print(f'{model_choice} ile {epochs} epoch için eğitim yapılıyor...')
            model = build_model(best_hyperparams_cstoa, base_model_choice=model_choice)
            early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
            model, history = train_model(model, data, labels, epochs=epochs, callbacks=[early_stopping])
            evaluate_model(model, data, labels)

            # Tahminler ve grafiksel analiz
            y_pred = model.predict(data)
            y_pred_classes = np.argmax(y_pred, axis=1)
            plot_metrics(labels, y_pred, y_pred_classes, history)

    print('Tüm eğitim ve değerlendirme süreçleri tamamlandı.')

Epoch 1/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 153s 6s/step - accuracy: 0.2583 - loss: 310.7368 - val_accuracy: 0.2600 - val_loss: 1459.5847
Epoch 2/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 114s 5s/step - accuracy: 0.2330 - loss: 1921.6926 - val_accuracy: 0.2600 - val_loss: 3076.4634
Epoch 3/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 160s 5s/step - accuracy: 0.2503 - loss: 3520.6140 - val_accuracy: 0.2600 - val_loss: 4692.2715
32/32 ━━━━━━━━━━━━━━━━━━━━ 115s 4s/step - accuracy: 0.2537 - loss: 4755.5630
Epoch 1/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 130s 5s/step - accuracy: 0.0371 - loss: 29.4058 - val_accuracy: 0.0000e+00 - val_loss: 123.3042
Epoch 2/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 132s 5s/step - accuracy: 0.0000e+00 - loss: 154.5547 - val_accuracy: 0.0000e+00 - val_loss: 250.5885
Epoch 3/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 121s 5s/step - accuracy: 0.0000e+00 - loss: 281.1298 - val_accuracy: 0.0000e+00 - val_loss: 377.3668
32/32 ━━━━━━━━━━━━━━━━━━━━ 117s 4s/step - accuracy: 0.0000e+00 - loss: 456.4598
Epoch 1/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 15